In [1]:
import numpy as np
from cogent3 import get_app, open_data_store
from cogent3.app.composable import define_app, NotCompleted
from cogent3.maths.measure import jsd
import scipy.stats as stats
from scipy.spatial.distance import correlation
import plotly.express as px
import plotly.graph_objects as go

In [2]:
from clock_project.simulation.wts import taxonomic_triple_simulation, three_seq_simulation_star_shape, generate_ancestor


In [3]:
import json
with open ('../ensembl_genome_analysis/matrix_full.json', 'r') as infile:
    matrices_dict = json.load(infile)

In [4]:
matrix_collection = []
for gene, matrices_list in matrices_dict.items():
    matrix_collection.extend(matrices_list)

In [5]:
Q_ingroup1, Q_ingroup2, Q_outgroup = matrices_dict['ENSG00000020129'][0].values()

In [6]:
Q_ingroup1_, Q_ingroup2_, Q_outgroup_ = matrices_dict['ENSG00000048707'][0].values()

In [7]:
p0 = [0.25, 0.25, 0.25, 0.25]

In [8]:
path1 = '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_model_fitting_350_threshold/ENSG00000007933/model_fitting_result/0.json'
load_json_app = get_app("load_json")
res = load_json_app(path1)

In [9]:

def get_seed_ancester_seq(pi_low, pi_high, len_short, len_long):
    seq_ls = generate_ancestor(len_short, pi_low)
    seq_ll = generate_ancestor(len_long, pi_low)
    seq_hs = generate_ancestor(len_short, pi_high)
    seq_hl = generate_ancestor(len_long, pi_high)
    seqs = {'low_short': seq_ls, 'high_short': seq_hs, 'low_long': seq_ll, 'high_long': seq_hl}
    return seqs


In [10]:
# from cogent3 import open_data_store
# def simulate_taxanomic_triples(ancester_seq, Q_collection, pi, seed, t, path_to_dir):
#     length = len(ancester_seq)
#     for matrix_dict in Q_collection:
#         Q_ingroup1, Q_ingroup2, Q_outgroup = matrices_dict.values()
#         aln, ens = taxanomic_triple_simulation(p0, np.array(Q_ingroup1), np.array(Q_ingroup2), np.array(Q_outgroup), 0, t, length, 1, 123, ancester_seq)
#         path_to_dir = '/Users/gulugulu/Desktop/honours/data_local/bootstrapping_test_non'
#         out_dstore = open_data_store(path_to_dir, mode="w", suffix="json")
#         write_json_app = get_app("write_json", data_store=out_dstore)
#         write_json_app(aln, identifier = Q_collection.index(matrix_dict))






In [15]:
ances_seq = generate_ancestor(300, p0)

In [16]:
path_to_dir = '/Users/gulugulu/Desktop/honours/data_local/simulated_triple_aln'
out_dstore = open_data_store(path_to_dir, mode="w", suffix="json")
write_json_app = get_app("write_json", data_store=out_dstore)
Q_ingroup1, Q_ingroup2, Q_outgroup = matrix_collection[0].values()
aln, ens = taxonomic_triple_simulation(p0, np.array(Q_ingroup1), np.array(Q_ingroup2), np.array(Q_outgroup), 0, 1, 1000, 1, 123, ances_seq)
# Open a data store for each simulation
write_json_app(aln, identifier=f'{matrix_collection.index(matrix_collection[0])}.json')

DataMember(data_store=/Users/gulugulu/Desktop/honours/data_local/simulated_triple_aln, unique_id=0.json)

In [17]:
import itertools

def generate_combinations(time_list, length_list, pi_list):
    # Create all combinations of time, length, and pi
    combinations = itertools.product(time_list, length_list, pi_list)
    return list(combinations)

# Example lists
time_list = [0, 1]
length_list = [100, 1000]
pi_list = ['pi1', 'pi2']

# Generate and print all combinations
combinations = generate_combinations(time_list, length_list, pi_list)


In [49]:
aln1, ens1 = taxonomic_triple_simulation(p0, np.array(Q_ingroup1), np.array(Q_ingroup2), np.array(Q_outgroup), 0, 1, 300, 1, 123, ances_seq)

In [39]:
aln2, ens2= taxonomic_triple_simulation(p0, np.array(Q_ingroup1), np.array(Q_ingroup2), np.array(Q_outgroup_), 0, 1, 300, 1, 123, ances_seq)

Model fitting for all the simulated alignment

In [11]:
from cogent3.app.typing import AlignedSeqsType, SerialisableType, IdentifierType
from cogent3.app.composable import define_app
from cogent3 import get_app, open_data_store

@define_app
def model_fitting(aln: AlignedSeqsType, opt_args = None) -> SerialisableType:
    tree="('outgroup_edge3',('ingroup_edge1','ingroup_edge2')'internal_node');"
    GN_model = get_app("model", "GN", 
                    tree=tree,
                    opt_args=opt_args,
                    # unique_trees=True,
                    # lf_args=dict(discrete_edges=[['outgroup_edge3']]),
                    time_het="max",
                    optimise_motif_probs=True)
    res = GN_model(aln)
    return res

model_fitting_app = model_fitting()

load_json_app = get_app("load_json")

@define_app
def customised_load_json(DataMember: IdentifierType) -> AlignedSeqsType:
    unique_id = DataMember.unique_id
    aln = load_json_app(DataMember)
    aln.source = unique_id
    return aln

def get_id(result):
    return result.alignment.source


load_json_app_customised = customised_load_json()

outdir = '/Users/gulugulu/Desktop/honours/data_local/simulated_triple_model_fitting/aln_high_5000_1'
out_dstore = open_data_store(outdir, mode="w", suffix="json")
write_json_app = get_app("write_json", data_store=out_dstore, id_from_source = get_id)

simulated_aln_high_5000_2 = '/Users/gulugulu/Desktop/honours/data_local/simulated_triple_aln/aln_low_5000_2'
input_data_store_simulated_aln = open_data_store(simulated_aln_high_5000_2, suffix= 'json')



In [12]:
app = load_json_app_customised+ model_fitting_app
res = app(input_data_store_simulated_aln[0])

In [13]:
aln = load_json_app(input_data_store_simulated_aln[0])

In [14]:
from clock_project.simulation.wts import calculate_non_stationarity
from cogent3.maths.measure import jsd
def get_triads_info(aln, res):
    seq_names = aln.names
    matrices = {n: res.lf.get_rate_matrix_for_edge(n, calibrated=True) for n in seq_names}
    sub_ens_tree = res.lf.get_ens_tree()
    lca_node = sub_ens_tree.lowest_common_ancestor(['ingroup_edge1', 'ingroup_edge2']).get_node_names()[0]
    ingroup_seqs = aln.take_seqs(['ingroup_edge1', 'ingroup_edge2'])
    nuc_freqs = ingroup_seqs.probs_per_seq()
    nuc_freqs1 = nuc_freqs['ingroup_edge1']
    nuc_freqs2 = nuc_freqs['ingroup_edge2']
    ingroup_jsd = jsd(nuc_freqs1, nuc_freqs2)
    distribution_internal_root = list(res.lf.get_motif_probs_by_node()[lca_node])
    jsd1 = jsd(nuc_freqs1, distribution_internal_root)
    jsd2 = jsd(nuc_freqs2, distribution_internal_root)
    distribution_root = list(res.lf.get_motif_probs_by_node()['root'])
    ens_dict = {n: sub_ens_tree.to_rich_dict()['edge_attributes'][n]['length'] for n in seq_names}
    ens_difference = abs(ens_dict['ingroup_edge1']-ens_dict['ingroup_edge2'])
    jsd_dict = {'Ingroup_JSD': ingroup_jsd, 'JSD_difference': {'ingroup_edge1': jsd1, 'ingroup_edge2': jsd2}}
    nabla_dict = {}
    for species in ['ingroup_edge1', 'ingroup_edge2']:
        nabla_dict[species] = calculate_non_stationarity(distribution_internal_root, matrices[species], ens_dict[species])
    nabla_dict['outgroup_edge3'] = calculate_non_stationarity(distribution_root, matrices['outgroup_edge3'], ens_dict['outgroup_edge3'])
    shortest_ens = min(ens_dict.values())  
    matrices_list = {n: matrices[n].to_array().tolist() for n in seq_names}
    distributions = {'internal_node': distribution_internal_root, 'root': distribution_root, 'ingroup1': list(nuc_freqs1), 'ingroup2': list(nuc_freqs2)}
    triad_data = {'triads_species_names': seq_names, 'triads_info_small_tree': 
        {'ens_difference': ens_difference,
        'shortest_ingroup_ens': shortest_ens,
        'ens': ens_dict,
        'jsd_dict': jsd_dict,
        'nuc_freqs_dict': distributions,
        'nabla_values': nabla_dict,
        'matrices': matrices_list}}
    return triad_data

In [15]:
aln.probs_per_seq()

ingroup_edge1,0.2330,0.2552,0.2452,0.2666
ingroup_edge2,0.1882,0.2884,0.3106,0.2128
outgroup_edge3,0.2402,0.2782,0.2466,0.2350


In [16]:
res.lf.get_motif_probs_by_node()

root,0.2591,0.2454,0.2640,0.2314
outgroup_edge3,0.2350,0.2781,0.2402,0.2466
internal_node,0.2544,0.2539,0.2483,0.2434
ingroup_edge1,0.2667,0.2550,0.2331,0.2452
ingroup_edge2,0.2127,0.2882,0.1884,0.3107


In [17]:
get_triads_info(aln, res)

{'triads_species_names': ['ingroup_edge1', 'ingroup_edge2', 'outgroup_edge3'],
 'triads_info_small_tree': {'ens_difference': 0.050871592127046894,
  'shortest_ingroup_ens': 0.058656513867625805,
  'ens': {'ingroup_edge1': 0.3207321309779797,
   'ingroup_edge2': 0.2698605388509328,
   'outgroup_edge3': 0.058656513867625805},
  'jsd_dict': {'Ingroup_JSD': 0.007417881565678686,
   'JSD_difference': {'ingroup_edge1': 0.0007275389796452814,
    'ingroup_edge2': 0.0076309657433017986}},
  'nuc_freqs_dict': {'internal_node': [0.25440807383737934,
    0.2538715646102941,
    0.24829801744581037,
    0.24342234410651614],
   'root': [0.25912521694994767,
    0.24537883928135276,
    0.2640478616552446,
    0.23144808211345497],
   'ingroup1': [0.233, 0.2552, 0.2452, 0.2666],
   'ingroup2': [0.1882, 0.2884, 0.3106, 0.2128]},
  'nabla_values': {'ingroup_edge1': 0.04886884516246603,
   'ingroup_edge2': 0.28729330139493847,
   'outgroup_edge3': 0.7826931074765656},
  'matrices': {'ingroup_edge1': [

In [12]:
path = '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_model_fitting_350_threshold/ENSG00000007933/model_fitting_result/0.json'
res_1 = load_json_app(path)

In [70]:
res_1.get_ens_tree().get_figure()

In [14]:
res_1.get_rate_matrix_for_edge("Ma's_night_monkey", calibrated=True)

T,-0.7648,0.5499,0.1486,0.0663
C,1.0654,-1.4139,0.1575,0.1911
A,0.2451,0.0000,-0.7942,0.5490
G,0.0473,0.1964,0.5971,-0.8408


In [18]:
res_1.get_motif_probs_by_node()['Euarchontoglires']

0.2489,0.3162,0.2075,0.2274


In [19]:
aln1.probs_per_seq()

NameError: name 'aln1' is not defined

In [15]:
res_1.get_all_rate_matrices()

{('Euarchontoglires',): =============================================
            T          C          A          G
 ---------------------------------------------
 T    -1.1225     0.0000     1.1005     0.0220
 C     0.4227    -1.0058     0.5831     0.0000
 A     0.0010     0.0173    -0.0182     0.0000
 G     0.2226     0.2836     1.0894    -1.5955
 ---------------------------------------------,
 ("Ma's_night_monkey",): =============================================
            T          C          A          G
 ---------------------------------------------
 T    -0.7648     0.5499     0.1486     0.0663
 C     1.0654    -1.4139     0.1575     0.1911
 A     0.2451     0.0000    -0.7942     0.5490
 G     0.0473     0.1964     0.5971    -0.8408
 ---------------------------------------------,
 ('Siberian_musk_deer',): =============================================
            T          C          A          G
 ---------------------------------------------
 T    -1.1390     0.6766     0.33

In [72]:
big_tree_result_path = '/Users/gulugulu/repos/PuningAnalysis/results/output_data/model_fitting_result_350_threshold/ENSG00000007933.json'
big_tree_result = load_json_app(big_tree_result_path)
big_tree_result.get_ens_tree().get_figure(height=1200, width=1200)

In [79]:
from clock_project.simulation.wts import calculate_non_stationarity

def get_triads_info(path):
    aln = load_json_app_customised(path)
    model_fitting_result = model_fitting_app(aln)
    seq_names = aln.names
    matrices = {n: model_fitting_result.lf.get_rate_matrix_for_edge(n, calibrated=True) for n in seq_names}
    sub_ens_tree = model_fitting_result.lf.get_ens_tree()
    lca_node = sub_ens_tree.lowest_common_ancestor(['ingroup_edge1', 'ingroup_edge2']).get_node_names()[0]
    ingroup_seqs = aln.take_seqs(['ingroup_edge1', 'ingroup_edge2'])
    nuc_freqs = ingroup_seqs.probs_per_seq()
    nuc_freqs1 = nuc_freqs['ingroup_edge1']
    nuc_freqs2 = nuc_freqs['ingroup_edge2']
    ingroup_jsd = jsd(nuc_freqs1, nuc_freqs2)
    distribution_internal_root = list(model_fitting_result.lf.get_motif_probs_by_node()[lca_node])
    jsd1 = jsd(nuc_freqs1, distribution_internal_root)
    jsd2 = jsd(nuc_freqs2, distribution_internal_root)
    distribution_root = list(model_fitting_result.lf.get_motif_probs_by_node()['root'])
    ens_dict = {n: sub_ens_tree.to_rich_dict()['edge_attributes'][n]['length'] for n in seq_names}
    ens_difference = abs(ens_dict['ingroup_edge1']-ens_dict['ingroup_edge2'])
    jsd_dict = {'Ingroup_JSD': ingroup_jsd, 'JSD_difference': {'ingroup_edge1': jsd1, 'ingroup_edge2': jsd2}}
    nabla_dict = {}
    for species in ['ingroup_edge1', 'ingroup_edge2']:
        nabla_dict[species] = calculate_non_stationarity(distribution_internal_root, matrices[species], ens_dict[species])
    nabla_dict['outgroup_edge3'] = calculate_non_stationarity(distribution_root, matrices['outgroup_edge3'], ens_dict['outgroup_edge3'])
    shortest_ens = min(ens_dict.values())  
    matrices_list = {n: matrices[n].to_array().tolist() for n in seq_names}
    distributions = {'internal_node': distribution_internal_root, 'root': distribution_root, 'ingroup1': list(nuc_freqs1), 'ingroup2': list(nuc_freqs2)}
    triad_data = {'triads_species_names': seq_names, 'triads_info_small_tree': 
        {'ens_difference': ens_difference,
        'shortest_ingroup_ens': shortest_ens,
        'ens': ens_dict,
        'jsd_dict': jsd_dict,
        'nuc_freqs_dict': distributions,
        'nabla_values': nabla_dict,
        'matrices': matrices_list}}
    return triad_data, model_fitting_result

In [80]:
info, res = get_triads_info(input_data_store_simulated_aln[0]) 

In [1]:
res.alignment.source.split('.')[0]

NameError: name 'res' is not defined

In [1]:
import glob
import os
path = '/Users/gulugulu/Desktop/honours/data_local/simulated_triple_aln'
triads_alignment_paths = glob.glob(os.path.join(path, "*.json"))

In [10]:
paths = glob.glob(os.path.join(path, '*/'))